In [1]:
import pandas as pd
import re
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions


### Reading Data

In [2]:
#dataset with news between 2008 and 2016 from Rosario12
df = pd.read_json("data/scraped_content.json", lines=True)
df['date'] = df['date'].dt.strftime('%Y/%m/%d')


In [3]:
df_exploded = df.explode('content').reset_index(drop=True)

In [4]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="paraphrase-multilingual-mpnet-base-v2")


In [ ]:
# Initialize Chroma client with persistence
client = chromadb.PersistentClient(path="data/chromastorage")

# Delete the collection if it exists
client.delete_collection(name="rag_2008_2011")

# Create a Chroma collection (persistent)
collection = client.create_collection(
    name="rag_2008_2011",
    embedding_function=sentence_transformer_ef  # Your sentence transformer embedding function
)

# Iterate through the rows and insert embeddings with metadata
for idx, row in df_exploded.iterrows():
    # Ensure content is wrapped in a list
    documents = [row['content']]

    # Define metadata
    metadata = {
        "title": row['title'] if pd.notna(row['title']) else "No Title",
        "link": row['link'] if pd.notna(row['link']) else "No Link",
        "date": row['date'] if pd.notna(row['date']) else "No Date"
    }

    # Add to the collection
    collection.add(
        documents=documents,  # List of content
        metadatas=[metadata],  # Corresponding metadata
        ids=[str(idx)]         # Use index or another unique identifier
    )

Vectorizar con modelo preentrenado (ver posibilidad de hacerlo con otro específicamente entrenado en español)

In [33]:
query = "¿por dónde ingresa la droga a la Argentina?"

result = collection.query(query_texts=[query], n_results=5, include=["metadatas", "documents"])

result

{'ids': [['412', '102', '417', '403', '566']],
 'distances': None,
 'metadatas': [[{'date': '2011/08/14',
    'link': 'https://www.lacapital.com.ar/policiales/mitos-y-verdades-la-produccioacuten-cocaiacutena-cocinas-la-ciudad-n412063.html',
    'title': 'Mitos y verdades sobre la producción de cocaína en cocinas de la ciudad'},
   {'date': '2010/04/29',
    'link': 'https://www.ellitoral.com/index.php/diarios/2010/04/29/sucesos/SUCE-05.html',
    'title': 'Investigan el origen del paco secuestrado'},
   {'date': '2011/08/14',
    'link': 'https://www.lacapital.com.ar/policiales/mitos-y-verdades-la-produccioacuten-cocaiacutena-cocinas-la-ciudad-n412063.html',
    'title': 'Mitos y verdades sobre la producción de cocaína en cocinas de la ciudad'},
   {'date': '2011/08/06',
    'link': 'https://www.lacapital.com.ar/policiales/coacutemo-operaba-la-banda-narco-la-que-le-hallaron-50-kilos-cocaiacutena-n412657.html',
    'title': 'Cómo operaba la banda narco a la que le hallaron 50 kilos de c